<a href="https://colab.research.google.com/github/PaulToronto/AI-and-Machine-Learning-for-Coders---Book/blob/main/3_2_Building_a_CNN_to_Distinguish_Between_Horses_and_Humans.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Building a CNN to Distinguish Between Horses and Humans

## Imports

In [2]:
import urllib.request
import zipfile

## The Dataset

- In this dataset, the location of a feature is not always in the same place
- Contains over 1000 $300 \times 300$ images, approximately half each of horses and humans, rendered in different poses
- The images have different lighting, different poses, different skin tones, different zoom levels, different backgrounds
    - The classifier will have to determinee which parts of the image are important features for the classification, without being affected by the background

### The Keras ImageDataGenerator

- Many image based

In [13]:
url = 'https://storage.googleapis.com/learning-datasets/horse-or-human.zip'
training_dir = 'horse-or-human/training'
file_name = 'horses-or-human.zip'

In [12]:
urllib.request.urlretrieve(url, file_name)

('horses-or-human.zip', <http.client.HTTPMessage at 0x7f91f6753220>)

In [15]:
zip_ref = zipfile.ZipFile(file_name, 'r')
zip_ref.extractall(training_dir)
zip_ref.close()